In [24]:
import numpy as np
import matplotlib.pyplot as plt 
import numpy.random as ra
import pandas as pd
import seaborn as sns
import statistics
from scipy import signal
import numpy.linalg as la 
import HERA_hack

# Cross Correlations 

Here we compute the cross-correlation functions for a couple of preliminary maps. 

Recall: IRL the data the fields are **not isotropic** 

Okay with that out of the way, we can get to working on the cross-correlations. What we want to do in generate an ensemble of both noise and foreground realizaitons. Compute the dot product of the two vectors, this essensially will give you a measure of how correlated the fields are. If they both have consistenly positive values, then the result of their dot product will be positive. if they are both consitently negative then their result will be negative. If they are both randomly varied then their dot product will be averaged down near 0. 

Obviously one realization does not give you a full picture of the statitics because there are only a few pixels. So we will compute the dot product for many realizations. Plot the resultant dot products and perform stat analysis on them! 

In [3]:

########### DEFINE OBS AND NECESSARY VARS #########


freq_fid = 150

dishes = np.array([[0,0],[0,55],[30,30],[0,60],[2,55],[47,2],[45,23],[56,21],[30,115],[48,52],[100,100],[0,200],[115,30],[33,31],[49,11],[21,24],[25,6],[56,9],[12,13],[16,17],[38,17],[60,14],[26,28],[6,45],[3,37],[12,55],[200,0],[145,13],[134,65],[139,163]])

#observable corners of the sky [lat,long]
acorner = np.array([[120,270],[122,280],[120,280],[122,270]])

HERA = HERA_hack.telescope(dishes, latitude=-30, channel_width=1., Tsys=300, beam_width=3, beam = 'gaussian')

obs = HERA_hack.observation(HERA, 100, 100, 0.01,acorner,1, 0.2, norm = False, pbeam = False)

def generate_foregrounds():
    ############ SYNCHRO EMISSION ############

    alpha_0_syn = 2.8
    sigma_syn = 0.1
    Asyn = 335.4 #K

    pixel_flux_syn = []

    alpha_syn = np.random.normal(alpha_0_syn,sigma_syn,obs.Npix)

    for i in range(obs.Npix):
        flux = Asyn*(obs.freq/freq_fid)**(-alpha_syn[i])
        pixel_flux_syn.append(flux)




    ########### FREE FREE EMISSION ##########

    alpha_0_ff = 2.15
    sigma_ff = 0.01
    Aff = 33.5 #K

    pixel_flux_ff = []

    alpha_ff = np.random.normal(alpha_0_ff,sigma_ff,obs.Npix)

    for i in range(obs.Npix):
        flux = Aff*(obs.freq/freq_fid)**(-alpha_ff[i])
        pixel_flux_ff.append(flux)

    ########### UNRES POINT SOURCE ###########

    gamma = 1.75

    def dnds(s):
        return 4.*(s/880)**(-gamma)

    s = np.arange(8,100,1) #maybe make this an argument 
    n_sources = 10

    pdf = np.asarray([s,dnds(s)]) #0 is s, 1 is dnds
    prob = pdf[1]/float(sum(pdf[1]))
    cum_prob = np.cumsum(prob)

    def gen_fluxes(N):
        R = ra.uniform(0, 1, N)
        #Here we first find the bin interval that random number lies in min(cum_prob[])
        #then we find the flux who's index is that cum_prob
        #repat for all r in R
        return [int(s[np.argwhere(cum_prob == min(cum_prob[(cum_prob - r) > 0]))]) for r in R]

    alpha_0 = 2.5
    sigma = 0.5

    theta_res = np.abs(np.cos(obs.observable_coordinates()[1,0])-np.cos(obs.observable_coordinates()[0,0]))
    phi_res = obs.observable_coordinates()[30,1]- obs.observable_coordinates()[1,1]
    omega_pix = theta_res*phi_res
    factor = 1.4e-6*((obs.freq/freq_fid)**(-2))*(omega_pix**(-1))

    pixel_flux = []

    for i in range(obs.Npix):
        alpha = np.random.normal(alpha_0,sigma,n_sources)
        S_star = gen_fluxes(n_sources)
        sum_fluxes = 0 

        for i in range(n_sources-1):
            sum_fluxes += factor*S_star[i]*(obs.freq/freq_fid)**(-alpha[i])

        pixel_flux.append(sum_fluxes/n_sources)
        
    


    ########## TOTAL FG ################

    pixel_flux = np.asarray(pixel_flux)
    pixel_flux_ff = np.asarray(pixel_flux_ff)
    pixel_flux_syn = np.asarray(pixel_flux_syn)

    total_fg = pixel_flux + pixel_flux_ff + pixel_flux_syn
    
    
    ##### Bright Point Sources (PHHASE SPACE) ########
    
    #vis_ps = cos(2*np.pi*(bdotr)) - j*sin(2*np.pi*(bdotr)) 
    #(where b is computed for each baseline and r is the pos of point source)
    #depending on how the astro data is stored, may need to include some coord transformations
    #COORD TRANSFORM DO NOT FORGET. 
    

    return total_fg



In [36]:
obs.generate_map_noise()

nreal = 10

noise_1 = np.zeros((obs.Npix,nreal))

noise_2 = np.zeros((obs.Npix,nreal))

fg = np.zeros((obs.Npix,nreal))

cov = np.zeros(nreal)

mean_prods = np.zeros(nreal)

sigma_prod = np.zeros(nreal)

for i in range(nreal):
    noise_1= np.real(obs.generate_map_noise())
    noise_2= np.real(obs.generate_map_noise())
    fg = generate_foregrounds()
    cov[i] = np.dot(noise_1,fg)
    mean_prods[i] = (np.mean(noise_1))*(np.mean(fg))
    sigma_prod[i] = (statistics.stdev(noise_1))*(statistics.stdev(fg))

In [48]:
print(cov)

stacked = np.reshape((np.dstack((cov,mean_prods,sigma_prod))),(nreal,3))

[432.09889239 157.39811834 -10.65513621 111.33392815 -67.69936561
 170.82406484 374.45635912 583.93798564  25.25754236 -84.11364673]


In [49]:
stacked

array([[ 4.32098892e+02,  2.87577384e-01,  3.22683445e-02],
       [ 1.57398118e+02,  1.04895758e-01,  3.99020195e-02],
       [-1.06551362e+01, -7.05160642e-03,  3.31509765e-02],
       [ 1.11333928e+02,  7.32977335e-02,  3.16569303e-02],
       [-6.76993656e+01, -4.54304741e-02,  3.70749150e-02],
       [ 1.70824065e+02,  1.12504022e-01,  3.69077758e-02],
       [ 3.74456359e+02,  2.49159522e-01,  3.05142292e-02],
       [ 5.83937986e+02,  3.88600586e-01,  3.57977277e-02],
       [ 2.52575424e+01,  1.74056945e-02,  3.58039891e-02],
       [-8.41136467e+01, -5.55220217e-02,  2.94924188e-02]])

In [121]:
# %load gumdrop_corr
import numpy as np
import matplotlib.pyplot as plt 
import numpy.random as ra
import statistics 
from scipy import signal
import numpy.linalg as la 
import HERA_hack

#make sure that HERA hack is in the directory that you're running this code in! 
#Remember to SCP it over

########### DEFINE OBS AND NECESSARY VARS #########


freq_fid = 150

dishes = np.array([[0,0],[0,55],[30,30],[0,60],[2,55],[47,2],[45,23],[56,21],[30,115],[48,52],[100,100],[0,200],[115,30],[33,31],[49,11],[21,24],[25,6],[56,9],[12,13],[16,17],[38,17],[60,14],[26,28],[6,45],[3,37],[12,55],[200,0],[145,13],[134,65],[139,163]])

#observable corners of the sky [lat,long]
acorner = np.array([[120,270],[122,280],[120,280],[122,270]])

HERA = HERA_hack.telescope(dishes, latitude=-30, channel_width=1., Tsys=300, beam_width=3, beam = 'gaussian')

obs = HERA_hack.observation(HERA, 100, 100, 0.01,acorner,1, 0.2, norm = False, pbeam = False)

def generate_foregrounds():
############ SYNCHRO EMISSION ############

	alpha_0_syn = 2.8
	sigma_syn = 0.1
	Asyn = 335.4 #K

	pixel_flux_syn = []

	alpha_syn = np.random.normal(alpha_0_syn,sigma_syn,obs.Npix)

	for i in range(obs.Npix):
	    flux = Asyn*(obs.freq/freq_fid)**(-alpha_syn[i])
	    pixel_flux_syn.append(flux)




	########### FREE FREE EMISSION ##########

	alpha_0_ff = 2.15
	sigma_ff = 0.01
	Aff = 33.5 #K

	pixel_flux_ff = []

	alpha_ff = np.random.normal(alpha_0_ff,sigma_ff,obs.Npix)

	for i in range(obs.Npix):
	    flux = Aff*(obs.freq/freq_fid)**(-alpha_ff[i])
	    pixel_flux_ff.append(flux)

	########### UNRES POINT SOURCE ###########

	gamma = 1.75

	def dnds(s):
	    return 4.*(s/880)**(-gamma)

	s = np.arange(8,100,1) #maybe make this an argument 
	n_sources = 10

	pdf = np.asarray([s,dnds(s)]) #0 is s, 1 is dnds
	prob = pdf[1]/float(sum(pdf[1]))
	cum_prob = np.cumsum(prob)

	def gen_fluxes(N):
	    R = ra.uniform(0, 1, N)
	    #Here we first find the bin interval that random number lies in min(cum_prob[])
	    #then we find the flux who's index is that cum_prob
	    #repat for all r in R
	    return [int(s[np.argwhere(cum_prob == min(cum_prob[(cum_prob - r) > 0]))]) for r in R]

	alpha_0 = 2.5
	sigma = 0.5

	theta_res = np.abs(np.cos(obs.observable_coordinates()[1,0])-np.cos(obs.observable_coordinates()[0,0]))
	phi_res = obs.observable_coordinates()[30,1]- obs.observable_coordinates()[1,1]
	omega_pix = theta_res*phi_res
	factor = 1.4e-6*((obs.freq/freq_fid)**(-2))*(omega_pix**(-1))

	pixel_flux = []

	for i in range(obs.Npix):
	    alpha = np.random.normal(alpha_0,sigma,n_sources)
	    S_star = gen_fluxes(n_sources)
	    sum_fluxes = 0 

	    for i in range(n_sources-1):
	        sum_fluxes += factor*S_star[i]*(obs.freq/freq_fid)**(-alpha[i])
	    
	    pixel_flux.append(sum_fluxes/n_sources)


	########## TOTAL FG ################

	pixel_flux = np.asarray(pixel_flux)
	pixel_flux_ff = np.asarray(pixel_flux_ff)
	pixel_flux_syn = np.asarray(pixel_flux_syn)

	total_fg = pixel_flux + pixel_flux_ff + pixel_flux_syn

	return total_fg



############# X_CORR ############


nreal = 3

#cov_NFG = np.zeros(nreal)



#cov_NN = np.zeros(nreal)

#mean_prods_NFG = np.zeros(nreal)

#sigma_prod_NFG = np.zeros(nreal)

#mean_prods_NN = np.zeros(nreal)

#sigma_prod_NN = np.zeros(nreal)

corr_NFG = np.zeros(nreal)

corr_SN = np.zeros(nreal)

corr_NN = np.zeros(nreal)

    
for i in range(nreal):
    noise_1 = np.real(obs.generate_map_noise())
    noise_2 = np.real(obs.generate_map_noise())
    fg_1= generate_foregrounds()
    fg_2 = generate_foregrounds()
    n1 = noise_1 - np.mean(noise_1)
    n2 = noise_2 - np.mean(noise_2)
    nfg_1 = n1 + (fg_1 - np.mean(fg_1))
    nfg_2 = n2+ (fg_2 - np.mean(fg_2))

    norm_1 = np.sqrt(np.dot(n1,n1))
    norm_2 = np.sqrt(np.dot(n2,n2))
    norm_nfg_1 = np.sqrt(np.dot(nfg_1,nfg_1))
    norm_nfg_2 = np.sqrt(np.dot(nfg_2,nfg_2))
    
    cov_NFG = np.dot(nfg_1,nfg_2)
    corr_NFG[i] = cov_NFG /(norm_nfg_1*norm_nfg_2)

    cov_NN = np.dot(n1,n2)
    corr_NN[i] = cov_NN /(norm_1*norm_2)

    cov_SN = np.dot(n1,n1)
    corr_SN[i] = cov_SN /(norm_1*norm_1)
    
#     noise_1 = np.real(obs.generate_map_noise())
#     noise_2 = np.real(obs.generate_map_noise())
#     fg_1= generate_foregrounds()
#     fg_2 = generate_foregrounds()
    
#     print(fg_1)
#     print(fg_2)
#     print(noise_1)
#     print(noise_2)

#     nfg_1 = noise_1 + fg_1
#     nfg_2 = noise_2 + fg_2
    
#     print(nfg_1)
#     print(nfg_2)
#     assert False

#     norm_1 = np.sqrt(np.dot(noise_1,noise_1))
#     norm_2 = np.sqrt(np.dot(noise_2,noise_2))
#     norm_nfg_1 = np.sqrt(np.dot(nfg_1,nfg_1))
#     norm_nfg_2 = np.sqrt(np.dot(nfg_2,nfg_2))
    
#     cov_NFG = np.dot(nfg_1,nfg_2)
#     mean_prods_NFG = (np.mean(nfg_1))*(np.mean(nfg_2))
#     corr_NFG[i] = (cov_NFG - (mean_prods_NFG))/(norm_nfg_1*norm_nfg_2)
    
#     cov_NN = np.dot(noise_1,noise_2)
#     mean_prods_NN = (np.mean(noise_1))*(np.mean(noise_2))
#     corr_NN[i] = (cov_NN - (mean_prods_NN))/(norm_1*norm_2)
    
#     cov_SN = np.dot(noise_1,noise_1)
#     mean_prods_SN = (np.mean(noise_1))*(np.mean(noise_1))
#     corr_SN[i] = (cov_SN - (mean_prods_SN))/(norm_1*norm_1)
   
    



#     cov_NFG[i] = np.dot(noise_1,fg)
#     mean_prods_NFG[i] = (np.mean(noise_1))*(np.mean(fg))
#     sigma_prod_NFG[i] = (statistics.stdev(noise_1))*(statistics.stdev(fg))

#     cov_NN[i] = np.dot(noise_1,noise_2)
#     mean_prods_NN[i] = (np.mean(noise_1))*(np.mean(noise_2))
#     sigma_prod_NN[i] = (statistics.stdev(noise_1))*(statistics.stdev(noise_2))

# stacked_NFG = np.reshape((np.dstack((cov_NFG,mean_prods_NFG,sigma_prod_NFG))),(nreal,3))
# # stacked_NN = np.reshape((np.dstack((cov_NN,mean_prods_NN,sigma_prod_NN))),(nreal,3))


# #np.savetxt('noise_fg.txt',stacked_NN)

# np.savetxt('same_noise.txt', stacked_NFG)




In [117]:
x = 2*np.ones(10)
y = np.arange(-10,10,1)

x-2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [105]:
print(corr_NFG)

[0.9978624  0.99791029 0.99795892]


In [123]:
corr_NN

array([0.04363303, 0.04780113, 0.02240051])

In [122]:
corr_SN

array([1., 1., 1.])

In [124]:
corr_NFG

array([-0.01521259,  0.002039  , -0.00917932])